In [1]:
from dataloader import training_dataset, testing_dataset
import torch
from torch.utils.data import DataLoader
from vit_pytorch import ViT, SimpleViT
from torch import nn
from tqdm import tqdm
import os

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
training_data = DataLoader(training_dataset, batch_size=32, shuffle=True)
testing_data = DataLoader(testing_dataset, batch_size=32, shuffle=True)

In [4]:
vision_model = ViT(
    image_size=128 * 3,
    patch_size=32,
    num_classes=39,
    dim=512,
    depth=6,
    heads=16,
    mlp_dim=1024,
).to('cuda')

# vit_model = nn.Sequential(vision_model, nn.Sigmoid())

In [5]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(vision_model.parameters(), lr=1e-7)

In [6]:
def train_loop(dataloader, testing_data, model, loss_fn, optimizer, state={'loss': [], 'epochs': [], 'test_loss': []}, epochs=10, save_path='./saves'):
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    all_exps = [int(elm.replace('exp_', '')) if elm != '.ipynb_checkpoints' else -1 for elm in os.listdir(save_path)]

    current_num = max(all_exps) if (len(all_exps) > 0) else 1
    save_path = save_path + '/exp_' + str(current_num + 1)
    os.mkdir(save_path)

    model.to(device)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        loop = tqdm(dataloader, total=len(dataloader), leave=True)
        total_loss = 0

        for batch_idx, (data, targets) in enumerate(loop):
            data = data.to(device)
            targets = targets.to(device)

            # Forward pass
            scores = model(data)
            loss = loss_fn(scores, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update progress bar
            loop.set_postfix(loss=loss.item())

        testing_loss = 0
        for test in tqdm(testing_data):
            X, y = test
            with torch.no_grad():
                model.eval()

                score = model(X)
                testing_loss =+ loss_fn(score, y)

                model.train()

        print(f"Epoch {epoch + 1} average loss: {total_loss / len(dataloader)} with testing loss of {testing_loss / len(testing_data)}")

        state['loss'].append(total_loss / len(dataloader))
        state['epochs'].append(epoch + 1)
        state['test_loss'].append(testing_loss / len(testing_data))

        # Save the model after each epoch
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss / len(dataloader),
            'testing_loss': testing_loss / len(testing_data)
        }, f"{save_path}/model_epoch_{epoch + 1}.pth")

    print("Training complete!")

In [7]:
state = train_loop(training_data, testing_data, vision_model, loss_fn, optimizer, epochs=100)
torch.save(state, './stats.pth')

Epoch 1/100


100%|██████████| 13/13 [00:12<00:00,  1.00it/s]


Epoch 1 average loss: 0.788705300539732 with testing loss of 0.06024235114455223
Epoch 2/100


100%|██████████| 13/13 [00:12<00:00,  1.02it/s]


Epoch 2 average loss: 0.7749776365235448 with testing loss of 0.05804092437028885
Epoch 3/100


...

Epoch 27 average loss: 0.5028000492602587 with testing loss of 0.0402546226978302
Epoch 28/100


100%|██████████| 13/13 [00:12<00:00,  1.01it/s]


Epoch 28 average loss: 0.4945832509547472 with testing loss of 0.04065763205289841
Epoch 29/100


  2%|▏         | 1/64 [00:01<01:28,  1.40s/it, loss=0.484]


KeyboardInterrupt: 